# Sample Data Generation Notebook

##### Get usernames and categories

In [1]:
# Dependencies: pip install python-dotenv Flask Flask-SQLAlchemy
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from flask import Flask

# Load backend/.env
load_dotenv(".env")

# Initialize Flask + SQLAlchemy
from models import db, User, Category  # uses the User and Category table from MySQL db
app = Flask(__name__)
app.config.update(
    # SQLALCHEMY_DATABASE_URI=os.getenv("DATABASE_URL") or os.getenv("DATABASE_URL"),
    SQLALCHEMY_DATABASE_URI=os.getenv("SHARED_DATABASE_URL") or os.getenv("DATABASE_URL"),
    SQLALCHEMY_TRACK_MODIFICATIONS=False,
)
db.init_app(app)

# Query users and categories
usernames, categories = [], []
with app.app_context():
    usernames = [u.username for u in User.query.order_by(User.username).all()]
    categories = [c.name for c in Category.query.order_by(Category.name).all()]

print(f"Found {len(usernames)} users")
print(usernames)
print(f"Found {len(categories)} categories")
print(categories[:10])

Found 14 users
['admin', 'bellaffelipe', 'BillyGarcia', 'ChristineNguyen', 'd_trump', 'dollheartz', 'elon_musk', 'FrostyBite', 'Gigi', 'karen_poor_reviews', 'nthPerson', 'PerryLee', 'RobertSlay', 'techy']
Found 103 categories
['3d printers', 'accessories', 'accessory', 'ai', 'animals', 'apple', 'apps & games', 'arts', 'automotive', 'baby']


##### Print usernames and categories

In [2]:
for i, user in enumerate(usernames):
    print(f'User {i}: {user}')

print(f'Categories ({len(categories)}): {categories}')

User 0: admin
User 1: bellaffelipe
User 2: BillyGarcia
User 3: ChristineNguyen
User 4: d_trump
User 5: dollheartz
User 6: elon_musk
User 7: FrostyBite
User 8: Gigi
User 9: karen_poor_reviews
User 10: nthPerson
User 11: PerryLee
User 12: RobertSlay
User 13: techy
Categories (103): ['3d printers', 'accessories', 'accessory', 'ai', 'animals', 'apple', 'apps & games', 'arts', 'automotive', 'baby', 'bath', 'beauty', 'bedding', 'bikes', 'board games', 'books', 'bunnies', 'cameras', 'camping & hiking', 'car', 'car seats', 'cars', 'cell phone', 'cell phones', 'cellphone', 'cellphones', 'cleaning supplies', 'clothes', 'clothing', 'collectibles', 'computers', 'costumes', 'crafts', 'diy', 'drones', 'educational toys', 'electronics', 'entertainment', 'fishing', 'fitness', 'furniture', 'games', 'garden', 'gift cards', 'grocery', 'gym wear', 'handmade', 'hardware', 'headphones', 'health', 'home', 'home improvement', 'industrial', 'jewelry', 'keyboard', 'kitchen', 'lighting', 'luggage', 'luxury items

##### Generate sample data using OpenAI API
-- 2nd Version of Item Sample Generation Cell
<br>Instead of making a single call to the OpenAI API to generate items, this version batches the number of requested samples and collects them all in `generated_items`.

In [4]:

import os
import json
import random
import re
import time
from datetime import date
from decimal import Decimal, ROUND_HALF_UP
from openai import OpenAI

# Set OpenAI API key
load_dotenv(".env")
OPENAI_API_KEY = os.getenv('OPENAI_PERSONAL_KEY')

# Instantiate client
client = OpenAI(api_key=OPENAI_API_KEY, project='proj_tKtmc8cXcBHwbMO9uMLyAJum')  # Personal key, COMP440 Online Shop project

def generate_items_with_llm(usernames, categories, n=100, per_call=12, max_calls=500, sleep_s=0.25):
    # Helpers
    def norm_price(val):
        dec = Decimal(str(val)).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)
        return str(dec)

    def is_recent_iso(d):
        try:
            y, m, d0 = map(int, d.split("-"))
            _ = date(y, m, d0)
            return True
        except Exception:
            return False

    def normalize_title(t: str) -> str:
        t = t.lower().strip()
        t = re.sub(r"[^a-z0-9\s]+", "", t)
        t = re.sub(r"\s+", " ", t)
        return t[:255]

    def diversify_item(item, attempt):
        # Light, safe variations to reduce near-duplicates while keeping schema intact
        title = item["title"]
        desc = item["description"]

        suffixes = [" Pro", " Plus", " Max", " Mini", " Lite", " SE", " Edition", " Bundle", " Kit", " Set"]
        extras = [
            "Includes original packaging.",
            "Ships quickly and well-protected.",
            "Gently used and fully functional.",
            "Comes with a protective case.",
            "Clean and well-maintained.",
            "Limited-time price.",
            "Tested before listing.",
        ]
        # Vary title by appending a suffix not already present
        for sfx in random.sample(suffixes, k=min(3, len(suffixes))):
            if sfx.lower() not in title.lower() and len(title) + len(sfx) <= 255:
                title = title + sfx
                break

        # Optionally tweak description with a short factual line
        if len(desc.split(".")) <= 3:
            extra = random.choice(extras)
            if extra not in desc:
                desc = (desc.rstrip().rstrip(".") + ". " + extra).strip()

        item["title"] = title
        item["description"] = desc
        return item

    system_msg = (
        "You are a data generation assistant. "
        "Return ONLY compact JSON that strictly matches the requested schema. "
        "No commentary. No markdown. No extra keys. "
        "Ensure high diversity in titles and descriptions. Avoid near-duplicates."
    )

    base_prompt = (
        "Generate realistic sample items. "
        "Output strictly as JSON with this shape: "
        '{ "items": [ { "title": str, "description": str, "price": number, "date_posted": "YYYY-MM-DD", '
        '"posted_by": str, "categories": [str, ...] } ] }'
    )

    out = []
    usernames_set = set(usernames)
    categories_set = set(categories)
    used_keys = set()  # (norm_title, posted_by)

    calls = 0
    while len(out) < n and calls < max_calls:
        calls += 1
        batch_target = min(per_call, n - len(out))
        # Pass a rolling window of recently used titles to discourage repeats across calls
        avoid_titles = [i["title"] for i in out[-200:]]

        user_payload = {
            "instructions": {
                "count": batch_target,
                "rules": [
                    "Each item must use posted_by from the provided usernames list.",
                    "Each item must have categories as 1 to 3 distinct names from the provided categories list.",
                    "Do NOT include star_rating or image_url fields.",
                    "price must be a positive number with two decimals, typically 5.00 to 5000.00.",
                    "date_posted must be an ISO date (YYYY-MM-DD) within the last 120 days.",
                    "title should be short and realistic for an online marketplace.",
                    "description should be 1-2 sentences with realistic details.",
                    "Avoid titles in avoid_titles and avoid near-duplicate wording.",
                    "Vary brands/models/materials/colors/conditions to increase diversity."
                ],
            },
            "fields_schema": {
                "title": "string",
                "description": "string",
                "price": "number(2-decimals)",
                "date_posted": "YYYY-MM-DD",
                "posted_by": "string from usernames",
                "categories": "array of 1-3 strings from categories"
            },
            "usernames": usernames,
            "categories": categories,
            "avoid_titles": avoid_titles
        }

        try:
            resp = client.chat.completions.create(
                model="gpt-4o-mini",
                temperature=0.8,  # slightly higher for diversity
                messages=[
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": base_prompt},
                    {"role": "user", "content": json.dumps(user_payload)}
                ]
            )
            content = resp.choices[0].message.content.strip()
            data = json.loads(content)
            items = data.get("items", [])
        except Exception:
            items = []

        for raw in items:
            # Validate core fields
            if raw.get("posted_by") not in usernames_set:
                continue
            cats = raw.get("categories") or []
            cats = [c for c in cats if c in categories_set]
            if not (1 <= len(cats) <= 3):
                continue
            if not isinstance(raw.get("price"), (int, float, str)):
                continue
            if not (isinstance(raw.get("title"), str) and isinstance(raw.get("description"), str)):
                continue
            if not (isinstance(raw.get("date_posted"), str) and is_recent_iso(raw["date_posted"])):
                continue

            item = {
                "title": raw["title"].strip()[:255],
                "description": raw["description"].strip(),
                "price": norm_price(raw["price"]),
                "date_posted": raw["date_posted"],
                "posted_by": raw["posted_by"],
                "categories": cats
            }

            # Uniqueness: norm(title)+posted_by
            attempts = 0
            while attempts < 3:
                key = (normalize_title(item["title"]), item["posted_by"])
                if key not in used_keys:
                    break
                attempts += 1
                item = diversify_item(item, attempts)

            # Final check
            key = (normalize_title(item["title"]), item["posted_by"])
            if key in used_keys:
                continue

            out.append(item)
            used_keys.add(key)

            if len(out) >= n:
                break

        if len(out) < n:
            time.sleep(sleep_s)  # gentle throttle/backoff between calls

    return out

# Request as many unique items as you want (e.g., 100+)
generated_items = generate_items_with_llm(usernames, categories, n=100, per_call=12)
print(f"Generated {len(generated_items)} unique items (in {len({(i['posted_by'], i['title']) for i in generated_items})} unique title/seller pairs).")
for i, it in enumerate(generated_items[:5], 1):
    print(i, it)

Generated 100 unique items (in 100 unique title/seller pairs).
1 {'title': 'Vintage Camera', 'description': 'A classic 35mm film camera in excellent condition, perfect for photography enthusiasts.', 'price': '250.00', 'date_posted': '2023-09-15', 'posted_by': 'ChristineNguyen', 'categories': ['cameras', 'collectibles']}
2 {'title': 'Mountain Bike', 'description': 'High-performance mountain bike, lightly used, ideal for rugged terrains and outdoor adventures.', 'price': '500.00', 'date_posted': '2023-08-20', 'posted_by': 'BillyGarcia', 'categories': ['bikes', 'outdoor activities']}
3 {'title': 'Wireless Headphones', 'description': 'Bluetooth headphones with noise cancellation and long battery life, great for music lovers.', 'price': '89.99', 'date_posted': '2023-10-01', 'posted_by': 'FrostyBite', 'categories': ['electronics', 'headphones']}
4 {'title': 'Gaming Chair', 'description': 'Ergonomic gaming chair with adjustable features, designed for comfort during long gaming sessions.', 'pr

##### Build SQL to insert generated items into `item`

In [5]:
# Copy the output into MySQL Workbench and run.
from decimal import Decimal

def sql_quote(s: str) -> str:
    if s is None:
        return "NULL"
    s = str(s).replace("\\", "\\\\").replace("'", "''").replace("\n", " ")
    return f"'{s}'"

def dec2(val) -> str:
    return str(Decimal(str(val)).quantize(Decimal("0.01")))

lines = []
lines.append("-- Insert generated items")
lines.append("START TRANSACTION;")
for it in generated_items:
    title = sql_quote(it["title"])
    desc = sql_quote(it["description"])
    date_posted = sql_quote(it["date_posted"])  # 'YYYY-MM-DD'
    price = dec2(it["price"])                   # numeric literal, no quotes
    posted_by = sql_quote(it["posted_by"])
    lines.append(
        f"INSERT INTO railway.item (`title`, `description`, `date_posted`, `price`, `posted_by`) "
        # f"INSERT INTO comp440db.item (`title`, `description`, `date_posted`, `price`, `posted_by`) "
        f"VALUES ({title}, {desc}, {date_posted}, {price}, {posted_by});"
    )
lines.append("COMMIT;")

sql_insert_items = "\n".join(lines)
print(sql_insert_items)

-- Insert generated items
START TRANSACTION;
INSERT INTO railway.item (`title`, `description`, `date_posted`, `price`, `posted_by`) VALUES ('Vintage Camera', 'A classic 35mm film camera in excellent condition, perfect for photography enthusiasts.', '2023-09-15', 250.00, 'ChristineNguyen');
INSERT INTO railway.item (`title`, `description`, `date_posted`, `price`, `posted_by`) VALUES ('Mountain Bike', 'High-performance mountain bike, lightly used, ideal for rugged terrains and outdoor adventures.', '2023-08-20', 500.00, 'BillyGarcia');
INSERT INTO railway.item (`title`, `description`, `date_posted`, `price`, `posted_by`) VALUES ('Wireless Headphones', 'Bluetooth headphones with noise cancellation and long battery life, great for music lovers.', '2023-10-01', 89.99, 'FrostyBite');
INSERT INTO railway.item (`title`, `description`, `date_posted`, `price`, `posted_by`) VALUES ('Gaming Chair', 'Ergonomic gaming chair with adjustable features, designed for comfort during long gaming sessions.'

##### Build SQL to map categories in `item_category`

In [6]:
# Copy the output into MySQL Workbench and run after inserting items.
def sql_quote(s: str) -> str:
    if s is None:
        return "NULL"
    s = str(s).replace("\\", "\\\\").replace("'", "''").replace("\n", " ")
    return f"'{s}'"

lines = []
lines.append("-- Map items to categories (uses title + posted_by to find item_id)")
lines.append("START TRANSACTION;")
for it in generated_items:
    title = sql_quote(it["title"])
    posted_by = sql_quote(it["posted_by"])
    for cat in it["categories"]:
        cat_q = sql_quote(cat)
        # Use INSERT IGNORE to avoid duplicate PK (item_id, category_name)
        lines.append(
            "INSERT IGNORE INTO railway.item_category (`item_id`, `category_name`)\n"
            # "INSERT IGNORE INTO comp440db.item_category (`item_id`, `category_name`)\n"
            f"SELECT i.id, {cat_q} FROM `item` AS i\n"
            f"WHERE i.title = {title} AND i.posted_by = {posted_by}\n"
            "ORDER BY i.id DESC\n"
            "LIMIT 1;"
        )
lines.append("COMMIT;")

sql_insert_item_categories = "\n".join(lines)
print(sql_insert_item_categories)

-- Map items to categories (uses title + posted_by to find item_id)
START TRANSACTION;
INSERT IGNORE INTO railway.item_category (`item_id`, `category_name`)
SELECT i.id, 'cameras' FROM `item` AS i
WHERE i.title = 'Vintage Camera' AND i.posted_by = 'ChristineNguyen'
ORDER BY i.id DESC
LIMIT 1;
INSERT IGNORE INTO railway.item_category (`item_id`, `category_name`)
SELECT i.id, 'collectibles' FROM `item` AS i
WHERE i.title = 'Vintage Camera' AND i.posted_by = 'ChristineNguyen'
ORDER BY i.id DESC
LIMIT 1;
INSERT IGNORE INTO railway.item_category (`item_id`, `category_name`)
SELECT i.id, 'bikes' FROM `item` AS i
WHERE i.title = 'Mountain Bike' AND i.posted_by = 'BillyGarcia'
ORDER BY i.id DESC
LIMIT 1;
INSERT IGNORE INTO railway.item_category (`item_id`, `category_name`)
SELECT i.id, 'outdoor activities' FROM `item` AS i
WHERE i.title = 'Mountain Bike' AND i.posted_by = 'BillyGarcia'
ORDER BY i.id DESC
LIMIT 1;
INSERT IGNORE INTO railway.item_category (`item_id`, `category_name`)
SELECT i.id

##### Generate sample reviews for each generated item

In [ ]:
# Dependencies: random, datetime, openai (already installed earlier)
import os
import json
import random
import time
from datetime import date, datetime, timedelta

ALLOWED_SCORES = ["Excellent", "Good", "Fair", "Poor"]

def pick_score():
    # Weighted distribution: more positive than negative
    return random.choices(
        population=ALLOWED_SCORES,
        weights=[45, 30, 18, 7],
        k=1
    )[0]

def clamp_date(dmin: date, dmax: date) -> date:
    if dmin > dmax:
        return dmax
    return dmin

def random_date_between(dmin_iso: str, dmax: date | None = None) -> str:
    # dmin_iso is 'YYYY-MM-DD'; dmax defaults to today
    dmax = dmax or date.today()
    try:
        y, m, d = map(int, dmin_iso.split("-"))
        start = date(y, m, d)
    except Exception:
        start = date.today() - timedelta(days=90)
    start = clamp_date(start, dmax)
    days = (dmax - start).days
    r = start + timedelta(days=random.randint(0, max(0, days)))
    return r.isoformat()

def default_remark_for_score(score: str, title: str) -> str:
    # Fallback if LLM output cannot be parsed
    templates = {
        "Excellent": f"Absolutely love the {title}! Great quality and exceeded expectations.",
        "Good": f"The {title} works well and is a solid value. Satisfied with the purchase.",
        "Fair": f"The {title} is okay but has a few drawbacks. Gets the job done.",
        "Poor": f"Not happy with the {title}. Quality and performance were disappointing."
    }
    return templates.get(score, f"The {title} was acceptable overall.")

def generate_reviews_with_llm(generated_items, usernames, max_per_item=3, batch_size=10, retry_max=3, base_sleep=0.4):
    # Build planned reviews (choose reviewers and scores first)
    planned = []
    usernames_set = set(usernames)

    for item in generated_items:
        seller = item["posted_by"]
        candidates = [u for u in usernames if u != seller]
        if not candidates:
            continue
        count = random.randint(1, max_per_item)
        chosen_reviewers = random.sample(candidates, k=min(count, len(candidates)))
        for reviewer in chosen_reviewers:
            score = pick_score()
            # Review date on/after item.date_posted up to today
            review_date = random_date_between(item["date_posted"])
            planned.append({
                "item_title": item["title"],
                "item_posted_by": seller,
                "item_description": item.get("description", ""),
                "user_id": reviewer,
                "score": score,
                "review_date": review_date,
                "remark": None  # to be filled by LLM
            })

    if not planned:
        print("No planned reviews (not enough distinct users).")
        return []

    # Helper to request remarks with retry/backoff
    def request_remarks(entries):
        system_msg = (
            "You are a helpful assistant that writes short review remarks. "
            "Return ONLY compact JSON and nothing else."
        )
        user_req = {
            "instructions": [
                "For each entry, write a 1-2 sentence remark that matches the sentiment of its score:",
                "- Excellent: clearly positive",
                "- Good: positive with minor caveat at most",
                "- Fair: mixed/neutral with some critique",
                "- Poor: clearly negative",
                "Keep remarks realistic and concise. No emojis. No markdown."
            ],
            "return_shape": {
                "remarks": ["string, same order and length as entries"]
            },
            "entries": [
                {
                    "score": r["score"],
                    "title": r["item_title"],
                    "context": (r["item_description"][:180] if r.get("item_description") else "")
                } for r in entries
            ]
        }
        for attempt in range(retry_max):
            try:
                resp = client.chat.completions.create(
                    model="gpt-4o-mini",
                    temperature=0.6,
                    messages=[
                        {"role": "system", "content": system_msg},
                        {"role": "user", "content": "Write remarks for these entries. Output strictly JSON."},
                        {"role": "user", "content": json.dumps(user_req)}
                    ]
                )
                content = resp.choices[0].message.content.strip()
                data = json.loads(content)
                return data.get("remarks", [])
            except Exception:
                # Exponential backoff with jitter
                sleep_s = (base_sleep * (2 ** attempt)) + random.uniform(0, 0.25)
                time.sleep(sleep_s)
        return []

    # Use batches of review entries (not items). batch_size=5 => 5 remarks per call.
    out_reviews = []
    for i in range(0, len(planned), batch_size):
        batch = planned[i:i+batch_size]
        remarks = request_remarks(batch)

        # Merge remarks with batch, applying safe fallbacks
        for idx, review in enumerate(batch):
            title = review["item_title"]
            score = review["score"]
            remark = None
            if idx < len(remarks) and isinstance(remarks[idx], str):
                remark = remarks[idx].strip()
            if not remark:
                remark = default_remark_for_score(score, title)

            # Validate fields
            if review["score"] not in ALLOWED_SCORES:
                continue
            if review["user_id"] == review["item_posted_by"]:
                continue
            if review["user_id"] not in usernames_set:
                continue
            # Finalize
            out_reviews.append({
                "item_title": review["item_title"],
                "item_posted_by": review["item_posted_by"],
                "user_id": review["user_id"],
                "score": review["score"],
                "review_date": review["review_date"],  # YYYY-MM-DD
                "remark": remark
            })

        # Gentle pacing between calls
        time.sleep(0.15)

    # Enforce one-review-per-user-per-item locally (mirror DB unique constraint)
    seen = set()
    deduped = []
    for r in out_reviews:
        key = (r["user_id"], r["item_title"], r["item_posted_by"])
        if key in seen:
            continue
        seen.add(key)
        deduped.append(r)

    print(f"Prepared {len(deduped)} reviews for {len(generated_items)} items.")
    # Preview first few
    for k, r in enumerate(deduped[:5], 1):
        print(k, {**r, "remark": (r["remark"][:80] + "..." if len(r["remark"]) > 80 else r["remark"])})
    return deduped

generated_reviews = generate_reviews_with_llm(generated_items, usernames, max_per_item=3)

Prepared 196 reviews for 100 items.
1 {'item_title': 'Vintage Camera', 'item_posted_by': 'ChristineNguyen', 'user_id': 'nthPerson', 'score': 'Excellent', 'review_date': '2023-12-01', 'remark': 'An outstanding choice for photography enthusiasts, this vintage camera is in exc...'}
2 {'item_title': 'Vintage Camera', 'item_posted_by': 'ChristineNguyen', 'user_id': 'Gigi', 'score': 'Good', 'review_date': '2025-01-02', 'remark': 'A solid option for photography lovers, this vintage camera is in great shape, th...'}
3 {'item_title': 'Mountain Bike', 'item_posted_by': 'BillyGarcia', 'user_id': 'nthPerson', 'score': 'Excellent', 'review_date': '2024-09-10', 'remark': 'This mountain bike is exceptional, offering high performance and light use, maki...'}
4 {'item_title': 'Mountain Bike', 'item_posted_by': 'BillyGarcia', 'user_id': 'Gigi', 'score': 'Excellent', 'review_date': '2023-11-29', 'remark': 'An impressive high-performance mountain bike that is lightly used, ideal for any...'}
5 {'item_titl

#### Build SQL to insert generated reviews into `comp440db.review`

In [11]:
# Copy the output into MySQL Workbench and run AFTER inserting items.
from decimal import Decimal

def sql_quote(s: str) -> str:
    if s is None:
        return "NULL"
    s = str(s).replace("\\", "\\\\").replace("'", "''").replace("\n", " ")
    return f"'{s}'"

lines = []
lines.append("-- Insert generated reviews (resolve item_id by title + posted_by)")
lines.append("START TRANSACTION;")
for rv in generated_reviews:
    review_date = sql_quote(rv["review_date"])  # 'YYYY-MM-DD'
    score = sql_quote(rv["score"])              # Enum: Excellent/Good/Fair/Poor
    remark = sql_quote(rv["remark"])
    user_id = sql_quote(rv["user_id"])
    title_q = sql_quote(rv["item_title"])
    posted_by_q = sql_quote(rv["item_posted_by"])

    # Use INSERT IGNORE to avoid violating unique constraint (user_id, item_id) on reruns
    lines.append(
        "INSERT IGNORE INTO railway.review (`review_date`, `score`, `remark`, `user_id`, `item_id`)\n"
        # "INSERT IGNORE INTO comp440db.review (`review_date`, `score`, `remark`, `user_id`, `item_id`)\n"
        f"SELECT {review_date}, {score}, {remark}, {user_id}, i.id FROM railway.item AS i\n"
        # f"SELECT {review_date}, {score}, {remark}, {user_id}, i.id FROM comp440db.item AS i\n"
        f"WHERE i.title = {title_q} AND i.posted_by = {posted_by_q}\n"
        "ORDER BY i.id DESC\n"
        "LIMIT 1;"
    )
lines.append("COMMIT;")

sql_insert_reviews = "\n".join(lines)
print(sql_insert_reviews)

-- Insert generated reviews (resolve item_id by title + posted_by)
START TRANSACTION;
INSERT IGNORE INTO railway.review (`review_date`, `score`, `remark`, `user_id`, `item_id`)
SELECT '2023-12-01', 'Excellent', 'An outstanding choice for photography enthusiasts, this vintage camera is in excellent condition and ready for great shots.', 'nthPerson', i.id FROM railway.item AS i
WHERE i.title = 'Vintage Camera' AND i.posted_by = 'ChristineNguyen'
ORDER BY i.id DESC
LIMIT 1;
INSERT IGNORE INTO railway.review (`review_date`, `score`, `remark`, `user_id`, `item_id`)
SELECT '2025-01-02', 'Good', 'A solid option for photography lovers, this vintage camera is in great shape, though there may be minor considerations for some users.', 'Gigi', i.id FROM railway.item AS i
WHERE i.title = 'Vintage Camera' AND i.posted_by = 'ChristineNguyen'
ORDER BY i.id DESC
LIMIT 1;
INSERT IGNORE INTO railway.review (`review_date`, `score`, `remark`, `user_id`, `item_id`)
SELECT '2024-09-10', 'Excellent', 'This mo

##### Recalculate and persist star_rating for all items

In [12]:
# Assumes `app` was already created and `db` initialized earlier in the notebook.
from models import db, Item

with app.app_context():
    # Ensure session sees the latest DB state (e.g., after Workbench inserts)
    db.session.expire_all()

    total = 0
    updated = 0
    for item in Item.query.yield_per(200):
        total += 1
        new_rating = item.calculate_star_rating()  # computes average from existing reviews
        if item.star_rating != new_rating:
            item.star_rating = new_rating
            updated += 1

    db.session.commit()
    print(f"Recalculated star_rating for {total} items. Updated {updated} rows.")

    # Preview a few items
    sample = Item.query.order_by(Item.id.desc()).limit(5).all()
    for it in sample:
        print(f"{it.id}: {it.title} -> {it.star_rating}")

Recalculated star_rating for 121 items. Updated 100 rows.
121: Educational STEM Toy Set -> 4.17
120: Portable Bluetooth Projector -> 3.75
119: Camping Cooking Set -> 3.12
118: Vintage-Inspired Jewelry Box -> 4.58
117: Wireless Gaming Router -> 5.0


##### Generate Follower/Follow Sample Data

In [14]:
import random

# If usernames isn't defined for any reason, re-query
try:
    assert usernames and isinstance(usernames, list)
except Exception:
    from flask import Flask
    from models import db, User
    app = Flask(__name__)
    app.config.update(
        SQLALCHEMY_DATABASE_URI=os.getenv("SHARED_DATABASE_URL") or os.getenv("DATABASE_URL"),
        SQLALCHEMY_TRACK_MODIFICATIONS=False,
    )
    db.init_app(app)
    with app.app_context():
        usernames = [u.username for u in User.query.order_by(User.username).all()]

N = len(usernames)
if N < 2:
    print("Not enough users to generate follows.")
else:
    k_min = min(3, max(0, N - 1))   # Minimum of 3 followers/follows for each user
    k_max = min(10, max(0, N - 1))  # Maximum of 10 followers/follows for each user

    idx_of = {u: i for i, u in enumerate(usernames)}
    follows = {u: set() for u in usernames}    # u -> set of followees
    in_deg  = {u: 0 for u in usernames}        # follower counts

    # Step 1: base ring(s) to guarantee everyone has at least k_min in/out
    # For d = 1..k_min, connect i -> (i + d) mod N
    for d in range(1, k_min + 1):
        for i, u in enumerate(usernames):
            v = usernames[(i + d) % N]
            if v != u and v not in follows[u]:
                follows[u].add(v)
                in_deg[v] += 1

    # Step 2: add random extras up to k_max for both in/out, respecting caps
    for u in usernames:
        cur_out = len(follows[u])
        extra_budget = max(0, k_max - cur_out)
        extra_out = random.randint(0, extra_budget) if extra_budget > 0 else 0

        trials = 0
        while extra_out > 0 and trials < 5 * N:
            trials += 1
            v = random.choice(usernames)
            if v == u:
                continue
            if v in follows[u]:
                continue
            if in_deg[v] >= k_max:
                continue
            follows[u].add(v)
            in_deg[v] += 1
            extra_out -= 1

    # Sanity checks: no self follows, no duplicates
    pairs = []
    seen = set()
    for u, outs in follows.items():
        for v in outs:
            if u == v:
                continue
            key = (u, v)
            if key in seen:
                continue
            seen.add(key)
            pairs.append(key)

    # Build artifact for later use
    generated_follows = [{"follower_id": u, "followee_id": v} for (u, v) in pairs]

    # Summary
    out_deg = {u: len(follows[u]) for u in usernames}
    min_out, max_out = min(out_deg.values()), max(out_deg.values())
    min_in, max_in   = min(in_deg.values()),  max(in_deg.values())
    print(f"Users: {N}")
    print(f"Follow pairs: {len(generated_follows)}")
    print(f"Out-degree (followees) range: {min_out}..{max_out} (target {k_min}..{k_max})")
    print(f"In-degree (followers) range: {min_in}..{max_in} (target {k_min}..{k_max})")
    print("Sample:", generated_follows[:5])

    # Emit SQL for MySQL Workbench
    def sql_quote(s: str) -> str:
        if s is None:
            return "NULL"
        s = str(s).replace("\\", "\\\\").replace("'", "''").replace("\n", " ")
        return f"'{s}'"

    lines = []
    lines.append("-- Insert generated follows (username->username). Avoids duplicates with INSERT IGNORE.")
    lines.append("START TRANSACTION;")
    for rel in generated_follows:
        follower = sql_quote(rel["follower_id"])
        followee = sql_quote(rel["followee_id"])
        lines.append(
            "INSERT IGNORE INTO railway.follow (`follower_username`, `user_username`) "
            f"VALUES ({follower}, {followee});"
            # "INSERT IGNORE INTO comp440db.follow (`follower_username`, `user_username`) "
            # f"VALUES ({follower}, {followee});"
        )
    lines.append("COMMIT;")

    sql_insert_follows = "\n".join(lines)
    print(sql_insert_follows)

Users: 14
Follow pairs: 84
Out-degree (followees) range: 4..9 (target 3..10)
In-degree (followers) range: 5..8 (target 3..10)
Sample: [{'follower_id': 'admin', 'followee_id': 'PerryLee'}, {'follower_id': 'admin', 'followee_id': 'elon_musk'}, {'follower_id': 'admin', 'followee_id': 'BillyGarcia'}, {'follower_id': 'admin', 'followee_id': 'FrostyBite'}, {'follower_id': 'admin', 'followee_id': 'bellaffelipe'}]
-- Insert generated follows (username->username). Avoids duplicates with INSERT IGNORE.
START TRANSACTION;
INSERT IGNORE INTO railway.follow (`follower_username`, `user_username`) VALUES ('admin', 'PerryLee');
INSERT IGNORE INTO railway.follow (`follower_username`, `user_username`) VALUES ('admin', 'elon_musk');
INSERT IGNORE INTO railway.follow (`follower_username`, `user_username`) VALUES ('admin', 'BillyGarcia');
INSERT IGNORE INTO railway.follow (`follower_username`, `user_username`) VALUES ('admin', 'FrostyBite');
INSERT IGNORE INTO railway.follow (`follower_username`, `user_use